In [2]:
import json
import requests
import time
import pandas as pd

In [1]:
path_dataset = "D:/datasets/flickr-30k-images"
path_captions = "D:/YandexDisk/datasets/flickr-30k-images-metadata.csv"
path_prepared_captions = "D:/flickr-30k-images-metadata.csv"

In [3]:
df_captions = pd.read_csv(path_captions, delimiter='|')
df_captions.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


## Перевод

Квоты — организационные ограничения, которые можно изменить по запросу в техническую поддержку.
Лимиты — технические ограничения, обусловленные особенностями архитектуры Yandex.Cloud. Изменение лимитов невозможно.

Yandex Translate. Квоты (4 апреля 2021):
Вызовов одного метода API в секунду – 20
Символов, отправленных на перевод, в час – 1 млн

In [134]:
IAM_TOKEN = "_____"
FOLDER_ID = "_____"

BEARER = "Bearer " + IAM_TOKEN
HEADERS = {
    "Content-type": "application/json",
    "Authorization": BEARER
}

def translate_with_yandex(text):
    data = {
        "folder_id": FOLDER_ID,
        "texts": text,
        "targetLanguageCode": "ru"
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate', headers=HEADERS, data=str(data).encode('utf-8'))
    json_text = json.loads(response.text)

    return json_text["translations"][0]["text"]

## Выбрать по миллиону символов

(1) 0 -- 16460
(2) 16461 -- 33234
(3) 33235 -- 50124
(4) 50125 -- 67337
(5) 67338 -- 80713

In [22]:
df_captions.iloc[-1]

image_name                                             998845445.jpg
 comment_number                                                    4
 comment           A man on a moored blue and white boat with hil...
Name: 153909, dtype: object

In [23]:
count_chars = 0
last_idx = -1


for idx in range(16460+1, 153909+1):
    count_chars += len(df_captions.iloc[idx, 2])
    if count_chars > 1000000:
        print(last_idx)
        break
    last_idx = idx

print(count_chars)

33234
1000103


In [27]:
count_chars = 0
last_idx = -1

for idx in range(16460+1, 33234+1):
    count_chars += len(df_captions.iloc[idx, 2])
    last_idx = idx

print(last_idx)
print(count_chars)

33234
999982


#### Первый миллион

In [18]:
df_captions_1 = df_captions[:16460]
df_captions_1.shape

(16460, 3)

In [19]:
df_captions_1.to_csv("D:/flickr-30k-images-metadata-1.csv", sep='|', encoding='utf-8', index=False)

#### Второй миллион

In [29]:
df_captions_2 = df_captions[16460+1:33234]
df_captions_2.shape

(16773, 3)

In [30]:
df_captions_2.to_csv("D:/flickr-30k-images-metadata-2.csv", sep='|', encoding='utf-8', index=False)

#### Третий миллион

In [40]:
df_captions_3 = df_captions[33235:50124]
df_captions_3.shape

(16889, 3)

In [41]:
df_captions_3.to_csv("D:/flickr-30k-images-metadata-3.csv", sep='|', encoding='utf-8', index=False)

#### Четвёртый миллион

In [54]:
df_captions_4 = df_captions[50125:67337]
df_captions_4.shape

(17212, 3)

In [56]:
df_captions_4.to_csv("D:/flickr-30k-images-metadata-4.csv", sep='|', encoding='utf-8', index=False)

#### Пятый миллион

In [7]:
df_captions_5 = df_captions[67338:80713]
df_captions_5.shape

(13375, 3)

In [8]:
df_captions_5.to_csv("D:/flickr-30k-images-metadata-5.csv", sep='|', encoding='utf-8', index=False)

## 20 запросов в секунду

In [31]:
def fake_translate(text):
    return text[::-1]

In [32]:
fake_translate("Привет")

'тевирП'

In [108]:
df_for_fake = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-for-fake.csv", delimiter='|')
df_for_fake.shape

(153910, 3)

In [110]:
df_for_fake.head(6)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together
5,10002456.jpg,0,Several men in hard hats are operating a giant...


In [124]:
def fake_requesting():
    start_idx = 0

    for idx in range(20, 16460+1, 20):
        start_time = time.time()

        df_for_twenty = df_for_fake[start_idx:idx]
        start_idx = idx

        for idx_for_twenty in df_for_twenty.index:
            df_for_twenty.at[idx_for_twenty, " comment"] = fake_translate(df_for_twenty.at[idx_for_twenty, " comment"])

        time_difference = time.time() - start_time

        if time_difference < 1.0:
            time.sleep(1.1 - time_difference)

In [113]:
%%time
fake_requesting()

Wall time: 15min 11s


In [135]:
translate_with_yandex("cloud")

'облако'

## Перевести первый миллион символов

In [132]:
df1 = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-1.csv", delimiter='|')
df1.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


In [21]:
def request_yandex_translate(data, col_name, start_index, end_index):
    start = start_index
    count_iters = 0

    for idx in range(start_index, end_index+1, 20):
        start_time = time.time()

        df_twenty = data[start:idx]
        start = idx

        for idx_twenty in df_twenty.index:
            df_twenty.at[idx_twenty, col_name] = translate_with_yandex(df_twenty.at[idx_twenty, col_name])

        count_iters += 20
        if count_iters > 999:
            data.to_csv("D:/temp.csv", sep='|', encoding='utf-8', index=False)
            count_iters = 0

        time_difference = time.time() - start_time

        if time_difference < 1.0:
            time.sleep(1.01 - time_difference)

In [14]:
%%time
request_yandex_translate()

Wall time: 1min 7s


## Перевести второй миллион символов

In [30]:
df2 = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-2.csv", delimiter='|')
df2.tail(5)

,image_name,comment_number,comment
16768,2547366477.jpg,4,A man and two children at the beach
16769,2547785434.jpg,0,A small and gray dog chases a large bull who i...
16770,2547785434.jpg,1,A brown and white cow jumps away from a black ...
16771,2547785434.jpg,2,A black and white dog is playing with a large ...
16772,2547785434.jpg,3,A small dog chasing after a bucking cow


In [31]:
%%time
request_yandex_translate(df2, " comment", 0, 16772)

Wall time: 1h 43min 26s


## Перевести третий миллион символов

In [47]:
df3 = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-3.csv", delimiter='|')
df3.tail(5)

,image_name,comment_number,comment
16884,2992217972.jpg,4,A man with tattoos at a costume party
16885,2992226950.jpg,0,A man on a bicycle rides down a small street a...
16886,2992226950.jpg,1,A poor looking man in a blue shirt is riding a...
16887,2992226950.jpg,2,A green pull wagon is parked under a tree in a...
16888,2992226950.jpg,3,"A man bicycling through a street, followed by ..."


In [48]:
%%time
request_yandex_translate(df3, " comment", 0, 16880)

Wall time: 1h 33min 30s


## Перевести четвёртый миллион символов

In [59]:
df4 = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-4.csv", delimiter='|')
df4.tail(5)

,image_name,comment_number,comment
17207,3398276602.jpg,2,A wheelchair bound person receiving a tip from...
17208,3398276602.jpg,3,Elderly man in wheelchair begging for handout
17209,3398276602.jpg,4,A man donates to a panhandler
17210,3398745929.jpg,0,Someone's legs are in frame running over a bea...
17211,3398745929.jpg,1,A person is running across the beach at the wa...


In [60]:
%%time
request_yandex_translate(df4, " comment", 0, 17200)

Wall time: 1h 57min 41s


## Перевести пятый миллион символов

In [9]:
df5 = pd.read_csv("D:/YandexDisk/datasets/flickr-30k-images-metadata-5.csv", delimiter='|')
df5.tail(5)

,image_name,comment_number,comment
13370,3707990914.jpg,3,A woman on a surfboard rides a wave
13371,3707990914.jpg,4,A surfer rides the wave
13372,3708172446.jpg,0,Two men are sitting on a bench looking at a wa...
13373,3708172446.jpg,1,Two men sit on a bench in front of a three tie...
13374,3708172446.jpg,2,Two people sit on a park bench looking at a fo...


In [16]:
%%time
request_yandex_translate(df5, " comment", 0, 17200)

Wall time: 1h 30min 48s


In [49]:
df3.to_csv("D:/russian-flickr-30k-images-metadata-3.csv", sep='|', encoding='utf-8', index=False)